In [106]:
import ipdb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

%matplotlib inline

names = ['google', 'state_names', 'stores', 'store_states', 'train', 'weather']  

files = ['googletrend.csv', 'state_names.csv', 'store.csv', 'store_states.csv', 'train.csv', 'weather.csv']  

dfs = {}

for name, file in zip(names, files):
    dfs[name] =  pd.read_csv(f'../../data/raw/{file}', low_memory=False)
    print("Got ", name, file)

Got  google googletrend.csv
Got  state_names state_names.csv
Got  stores store.csv
Got  store_states store_states.csv
Got  train train.csv
Got  weather weather.csv


In [107]:
google = dfs['google'].copy()
state_names = dfs['state_names'].copy()  # Done
stores = dfs['stores'].copy()
store_states = dfs['store_states'].copy()  # Done
train = dfs['train'].copy()  # Done
weather = dfs['weather'].copy()
all_dfs = [google, state_names, stores, store_states, train, weather]

# Create _raw copies for reference
google_raw = dfs['google']
state_names_raw = dfs['state_names']
stores_raw = dfs['stores']
store_states_raw = dfs['store_states']
train_raw = dfs['train']
weather_raw = dfs['weather']

In [108]:
# Fix spelling error in weather dataframe
if 'Min_VisibilitykM' in weather.columns:
    weather.rename(columns = {'Min_VisibilitykM':'Min_VisibilityKm'}, inplace=True)
if 'Min_DewpointC' in weather.columns:                                                                                                                                    
    weather.rename(columns={'Min_DewpointC': 'MinDew_pointC'}, inplace=True) 

def convert_to_snake_case(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    draft = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    return draft.replace('__', '_')

for df in all_dfs:
    col_list = list(df.columns)
    df.columns = pd.Index(map(convert_to_snake_case, col_list))

In [109]:
#info()   - Stores - CompetitionOpenSinceMonth fill with mean                                      
#   - Stores - CompetitionOpenSinceYear fill with mean                                       
#   - Stores - Promo2SinceWeek fill with zero                                                
#   - Stores - Promo2SinceYear fill with zero                                                
#   - Stores - PromoInterval fill with zero                                                  
#   - Weather - Max_VisibityKm fill with mean                                                
#   - Weather - Min_VisibitykM fill with mean                                                
#   - Weather - Mean_VisibityKm fill with mean                                               
#   - Weather - Max_Gust_SpeedKm_h fill with mean                                            
#   - Weather - CloudCover fill with mean                                                    
#   - Weather - CloudCover fill with string 'No events'    

stores['promo2_since_week'] = stores.promo2_since_week.fillna(stores.promo2_since_week.mean())
stores['promo2_since_year'] = stores.promo2_since_year.fillna(stores.promo2_since_year.mean())
stores['promo_interval'] = stores.promo_interval.fillna('None')
stores['competition_distance'] = stores.competition_distance.fillna(stores.competition_distance.mean())
stores['competition_open_since_month'] = stores.competition_open_since_month.fillna(stores.competition_open_since_month.mean())
stores['competition_open_since_year'] = stores.competition_open_since_year.fillna(stores.competition_open_since_year.mean())

weather['max_visibility_km'] = weather.max_visibility_km.fillna(weather.max_visibility_km.mean())
weather['min_visibility_km'] = weather.min_visibility_km.fillna(weather.min_visibility_km.mean())
weather['mean_visibility_km'] = weather.mean_visibility_km.fillna(weather.mean_visibility_km.mean())
weather['max_gust_speed_km_h'] = weather.max_gust_speed_km_h.fillna(weather.max_gust_speed_km_h.mean())
weather['cloud_cover'] = weather.cloud_cover.fillna(weather.cloud_cover.mean())
weather['events'] = weather.events.fillna('No Events')

# ADD OTHER CHECKS LIKE THIS
# stores_raw.Promo2SinceWeek.mean() == stores.Promo2SinceWeek.mean()

In [110]:
# Keep
google['state'] = google.file.str[-2:]
google.loc[google.state == 'NI', 'state'] = 'HB,NI'
google['week_start'] = pd.to_datetime(google.week.str[:10])

# Add date to google dataframe, aligned with starting week
start_date = pd.to_datetime(google.week.min()[:10])
end_date = pd.to_datetime(google.week.max()[-10:])

# create a new dataframe, week_lookup, listing all days in the                                                                                                                        
# period and their corresponding week                                                                                                                                                 
days = pd.date_range(start_date, end_date, freq='D')                                                                                                                                  
week_lookup = pd.DataFrame({'date': days})                                                                                                                                            
week_lookup['num'] = week_lookup['date'].dt.dayofweek                                                                                                                                 
week_lookup['offset'] = (week_lookup['num'] + 1) % 7                                                                                                                                  
week_lookup['Week_Start'] = week_lookup['date'] - pd.to_timedelta(week_lookup['offset'], unit='D')                                                                                                                                  
week_lookup.drop(['num', 'offset'], axis='columns', inplace=True)

#days = np.arange(start_date, end_date + pd.to_timedelta('1D'), pd.to_timedelta('1D'))
#weeks = np.arange(start_date, end_date + pd.to_timedelta('1D'), pd.to_timedelta('7D'))
#all_weeks = pd.Series(np.hstack([weeks for i in range(0,7)])) # 1036
#week_lookup = pd.DataFrame({'date': days, 'Week_Start': all_weeks})
google = week_lookup.merge(google, left_on='Week_Start', right_on='week_start')
google.drop(['file', 'Week_Start', 'week'], axis='columns', inplace=True) #len=14504
google = google[(google.date >= pd.to_datetime('2013-01-01')) & (google.date <= pd.to_datetime('2015-07-31'))]

# Merge order after changing files:
# store_states and state_names, on='State'
# that and weather, left_on='StateName', right_on='file'
# that and google, on=['Date', 'State'], ??how='outer'??
# that and stores, on='Store'
# that and train, on=['Date', 'Store'], ??how='outer'??
# all_dfs = [google, state_names, stores, store_states, train, weather]
# [, train]

# Keep and refactor
df = store_states.merge(state_names, on='state')
# After merging, drop both file and state_name - they are colinear with 'state'
df = df.merge(weather, left_on='state_name', right_on='file').drop(['file', 'state_name'], axis='columns')
df = df.merge(stores, on='store')
train['date'] = pd.to_datetime(train['date'])
df['date'] = pd.to_datetime(df['date'])
df = df.merge(train, on=['date', 'store'], how='outer')
# train.csv only goes through 2015-07-31
# See if doing this on google instead fixes this
# df = df[df.date <= pd.to_datetime('2015-07-31')]
df = df.merge(google, on=['date', 'state'])
df.loc[df.open.isnull(), 'open'] = 0
df.loc[df.sales.isnull(), 'sales'] = 0
df.loc[df.customers.isnull(), 'customers'] = 0
df.loc[df.promo.isnull(), 'promo'] = 0
df.loc[df.school_holiday.isnull(), 'school_holiday'] = 0
df.loc[df.state_holiday.isnull(), 'state_holiday'] = '0'
df['day_of_week'] = df.date.dt.dayofweek
df.loc[df.customers == 0, 'open'] = 0

In [120]:
df.to_csv('../../data/processed/wrangled_dataframe.csv')

In [118]:
df.head(30)
# df.date.nunique() * df.store.nunique() == len(df) # True
# df.isnull().any().any() # False
# df.info()
# df.state_holiday.unique()
# len(train[(train.sales == 0) & (train.open == 1)]) # 54

# DIG BACK INTO THIS -- THIS IS OK.  THERE ARE 33121 INSTANCES IN TRAIN.CSV WITHIN THE DATE RANGE
# WHERE A STORE DOES NOT HAVE A ROW OF DATA. IN THOSE INSTANCES, WE ASSIGNED OPEN = 0 AND SALES = 0.
# IN OTHER WORDS, WE'RE ASSUMING THAT WITHIN THE DATE RANGE, IF WE DIDN'T HAVE A ROW OF DATA FOR 
# THAT STORE, IT MEANS THAT THE STORE WAS CLOSED.
# len(train[(train.sales == 0) & (train.open == 0)]) # 172817
# len(df[(df.sales == 0) & (df.open == 0)]) # 205938
# 205938-172817 # 33121
# len(train) #1017209
# len(df) # 1050330
# 1050330 - 1017209 # 33121
# train.date.min() # Timestamp('2013-01-01 00:00:00')
# train.date.max() # Timestamp('2015-07-31 00:00:00')
# df.date.min() # Timestamp('2013-01-01 00:00:00')
# df.date.max() # Timestamp('2015-07-31 00:00:00')
# df.date.max() - df.date.min() # 941 days
# train.store.nunique() # 1115
# train.date.nunique() # 942
# 33121/1115 # 29.70493273542601
# 33121/942 # 35.160297239915074
# 1115 * 942 # 1050330

# DIG BACK INTO THIS -- THIS IS OK. THE LONGEST DISTANCE IS 46 MILES, AND ONLY TWO STORES HAVE A
# DISTANCE OF GREATER THAN 30 MILES. SEEMS HARD TO BELIEVE THAT A DRUGSTORE IN GERMANY WOULDN'T
# HAVE ANY COMPETITION WITHIN 45 MILES, BUT MAYBE THEY'RE USING A TIGHT DEFINITION OF COMPETITOR.
# IN ANY CASE, WE DON'T HAVE COMPELLING EVIDENCE THAT IT'S WRONG, PLUS IT'S NOT A LARGE ENOUGH
# GROUP NOR A LARGE ENOUGH OUTLIER TO DISTORT THE RESULTS.
# df.competition_distance.max() # 75860 -- this is 46 miles
len(df[df.competition_distance == 75860])
len(stores[stores.competition_distance == 75860]) # 1, store=453
len(stores[stores.competition_distance > 10000]) # 187 -- this is 6 miles
len(df[df.competition_distance > 10000])/942 # 187 -- this is 6 miles
# stores.competition_distance.isnull().any() # False
len(df[df.competition_distance > 50000])/942 # 187 -- this is 6 miles

# DIG BACK INTO THIS -- THERE ARE 54 STORES THAT ARE MARKED AS OPEN BUT HAVE NO SALES. 52 OF THEM
# ARE ALSO MARKED AS HAVING NO CUSTOMERS. THE TWO THAT HAVE CUSTOMERS BUT NO SALES HAVE 3 AND 5
# CUSTOMERS RESPECTIVELY. WHILE IT'S POSSIBLE THAT A STORE COULD BE OPEN BUT HAVE NO CUSTOMERS
# AND NO SALES, IT SEEMS MORE LIKELY TO BE A MISTAKE.  IN FACT, THERE ARE ONLY 6 OTHER INSTANCES
# OF A STORE BEING OPEN AND GETTING LESS THAN 50 CUSTOMERS (COUNTING THE TWO ALREADY MENTIONED).
# SO WE'LL MARK ANY STORE WITH 0 CUSTOMERS AS NOT BEING OPEN ON THAT DAY.
# len(df[(df.sales == 0) & (df.open == 1)]) # 54
# len(train[(train.sales == 0) & (train.open == 1)]) # 54
# len(df[(df.sales == 0) & (df.open == 1) & (df.customers != 0)]) # 2
# len(df[(df.open == 1) & (df.customers < 50)]) # 60
# len(df[(df.open == 1) & (df.customers == 0)]) # 52
# len(df[(df.open == 1) & (df.customers < 100)]) # 123
# len(df[(df.sales == 0) & (df.open == 1)]) # 54 originally -- 2 after the fix

# TEST THIS - OK, CHECKS OUT 
# (df.day_of_week == df.date.dt.dayofweek).all() # TRUE
# len(pd.DataFrame(df.groupby(['store', 'date']).size().rename('Freq'))) # 1050330
# df.groupby(['store', 'date']).ngroups #1050330
# len(pd.unique(df[['store', 'date']].values.ravel('K'))) # 2057
# 1115 + 942 # 2057
# len(pd.unique(df[['store', 'date']].values.ravel('K'))) # 2057

# CHECK THAT EACH (STORE, DATE) TUPLE IS UNIQUE
(df.groupby(['store', 'date']).size() == 1).all() # True

True

In [80]:
# google_raw.file.nunique() # 14
# google_raw.week.nunique() # 148
# len(google_raw) # 2072
# 148 * 14 # 2072
store_states.columns # 'store', 'state'
state_names.columns # 'state_name', 'state'
state_names_raw.columns # 'state_name', 'state'
# sorted(state_names.columns) == ['state', 'state_name'] 
# a = sorted(store_states.columns) # ['state', 'store']
# a = sorted(state_names.columns) # ['state', 'state_name']
a = sorted(weather.columns) # ['cloud_cover', 'date', 'dew_point_c', 'events', 'file', 'max_gust_speed_km_h',
 # 'max_humidity', 'max_sea_level_pressureh_pa', 'max_temperature_c', 'max_visibility_km', 'max_wind_speed_km_h',
 # 'mean_dew_point_c', 'mean_humidity', 'mean_sea_level_pressureh_pa', 'mean_temperature_c', 'mean_visibility_km',
 # 'mean_wind_speed_km_h', 'min_dewpoint_c', 'min_humidity', 'min_sea_level_pressureh_pa', 'min_temperature_c',
 # 'min_visibility_km', 'precipitationmm', 'state', 'state_name', 'store', 'wind_dir_degrees']
sorted(train.columns) # ['customers', 'date', 'day_of_week', 'open', 'promo', 'sales', 'school_holiday',
 # 'state_holiday', 'store']
sorted(stores.columns) # ['assortment', 'competition_distance', 'competition_open_since_month',
 # 'competition_open_since_year', 'promo2', 'promo2_since_week', 'promo2_since_year', 'promo_interval', 'store',
 # 'store_type']

['assortment',
 'competition_distance',
 'competition_open_since_month',
 'competition_open_since_year',
 'promo2',
 'promo2_since_week',
 'promo2_since_year',
 'promo_interval',
 'store',
 'store_type']

In [ ]:
#names = ['google', 'state_names', 'stores', 'store_states', 'train', 'weather']  

state_names_raw.info()

In [ ]:
fake = pd.DataFrame({'file': ['DE_BB', 'DE_BE', np.nan], 'week': ['2013-01-01 - 2013-01-07', '2013-01-15 - 2013-01-21', 'dropped'], 'trend': [98, 69, 12]})  
fake.dropna(axis='index', inplace=True) 
fake

In [ ]:
fake = pd.DataFrame({'file': ['DE_BB', 'DE_BE'], 'week': ['2013-01-06 - 2013-01-12', '2013-01-20 - 2013-01-26'], 'trend': [98, 69]})  
fake['state'] = fake.file.str[-2:]
fake.loc[fake.state == 'NI', 'state'] = 'HB,NI'
fake['week_start'] = pd.to_datetime(fake.week.str[:10])
start_date = pd.to_datetime(fake.week.min()[:10])
end_date = pd.to_datetime(fake.week.max()[-10:])
#days = np.arange(start_date, end_date + pd.to_timedelta('1D'), pd.to_timedelta('1D'))
days = pd.date_range(start_date, end_date, freq='D')
#weeks = np.arange(start_date, end_date + pd.to_timedelta('1D'), pd.to_timedelta('7D'))
#all_weeks = pd.Series(sorted(np.hstack([weeks for i in range(0,7)]))) # 1036
#week_lookup = pd.DataFrame({'date': days, 'Week_Start': all_weeks})
#fake = week_lookup.merge(fake, left_on='Week_Start', right_on='week_start')
#fake = fake.drop(['file', 'Week_Start', 'week'], axis='columns') #len=14504
#fake
#fake.Week_Start.dtype # dtype('<M8[ns]')
#fake.week_start.dtype # dtype('<M8[ns]')
a = pd.DataFrame({'date': days})
a['num'] = a['date'].dt.dayofweek
a['off2'] = (a['num'] + 1) % 7
a['Week_Start'] = a['date'] - pd.to_timedelta(a['off2'], unit='D')
a.drop(['num', 'off2'], axis='columns', inplace=True)
#a['week'] = a.loc[a.num != 6, a.day + pd.to_timedelta(f'{(a.num + 1) % 6}D')]
fake.merge(a, left_on='week_start', right_on='Week_Start')
# a['week'] = a['day'] - pd.offsets.Week(-1)
#a['week'] = a['day'] - pd.offsets.Week(weekday=6)
#a['week'] = a['day'] - pd.offsets.DateOffset(weekday=6)
#week_of_offsets = pd.to_timedelta([f'{i}D' for i in range(7)]).T
#num_of_weeks = len(a) // 7
#offsets = pd.Series(week_of_offsets for week in range(num_of_weeks))
#offsets.reshape(1, -1)
#start_string = str(start_date)[:10]
#end_string = str(end_date)[:10]
#range = pd.timedelta_range(start=start_string, end=end_string, freq='D')
#range
#pd.TimedeltaIndex.to_series(range)
#a['week'] = a['day'] - offsets
#a
#(pd.to_timedelta([f'{i}D' for i in range(6)])) + (pd.to_timedelta([f'{i}D' for i in range(6)]))
#pd.to_timedelta('1D')
# [f'{i}D' for i in range(10)]

In [ ]:
df.isnull().any()
df.info()
train.info()
#len(df)    # 1050330
#942 * 1115 # 1050330

In [ ]:
f'{word}'

In [86]:
for name in names:
    print(name, sorted(dfs[name].columns))

# date: google, train, weather
# state: google, state_names, store_states, weather
# store: store, store_states, train

google ['file', 'trend', 'week']
state_names ['State', 'StateName']
stores ['Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Store', 'StoreType']
store_states ['State', 'Store']
train ['Customers', 'Date', 'DayOfWeek', 'Open', 'Promo', 'Sales', 'SchoolHoliday', 'StateHoliday', 'Store']
weather ['CloudCover', 'Date', 'Dew_PointC', 'Events', 'Max_Gust_SpeedKm_h', 'Max_Humidity', 'Max_Sea_Level_PressurehPa', 'Max_TemperatureC', 'Max_VisibilityKm', 'Max_Wind_SpeedKm_h', 'MeanDew_PointC', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm', 'Mean_Wind_SpeedKm_h', 'Min_DewpointC', 'Min_Humidity', 'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Min_VisibilitykM', 'Precipitationmm', 'WindDirDegrees', 'file']


In [ ]:
# len(train) # 1017209
# train.Date.nunique() # 942
# train.Store.nunique() # 1115
1115 * 942 # 1050330 
1050330 - 1017209 # 33121
33121/942 # 35.16...
33121/1115 # 29.70...
# train.groupby('Store')['Date'].count().value_counts()
#    942    934
#    758    180
#    941      1
# 934 * 942 + 180 * 758 + 941 # 1017209

In [ ]:
train.promo.unique()
len(train[train.promo == 0]) #629129
len(train[train.promo == 1]) #388080
len(df.loc[df.school_holiday.isnull(), 'date'].unique()) # 185
len(df.loc[df.state_holiday.isnull(), 'date'].unique()) # 185
dates = np.sort(df.loc[df.state_holiday.isnull(), 'date'].unique())
#state_names_raw.columns
#df.state_holiday.unique()
#df.loc[df.state_holiday.isnull(), ['date', 'state', 'state_holiday']].head()
dates[:15]
#type(dates)
#df.loc[df.date.apply(lambda x: x in dates), ['date', 'state', 'state_holiday']].head()
df.loc[df.date == '2014-07-04T00:00:00.000000000', ['date', 'state', 'state_holiday']].head()
len(df[df.state_holiday.isnull()])
#df['date_state'] = df.date.dt.strftime('%Y-%m-%d') + df.state
df.date_state.tail()
df.loc[df.state_holiday.isnull(), 'date_state'].nunique() # 185
date_states = np.sort(df.loc[df.state_holiday.isnull(), 'date_state'].unique())
#df[df.date_state]
date_states[:15]
#df.loc[(df.date=='2014-07-03') & (df.state == 'BY') & (df.state_holiday.notnull()), 'state_holiday']
df.head()
df = df.set_index('date') #44
date_mask = dates
df[date_mask].head()
#df.index

In [ ]:
len(df)
# df.isnull().sum()
sorted(df.day_of_week.unique())

In [ ]:
#train.info()
#df.info()
train.head()
len(train) # 1017209
len(df) # 1050330
#train['date'] = pd.to_datetime(train['date'])
#df = df.merge(train, on=['date', 'store'])
len(df) # 1017209
train.date.nunique() # 942
train.store.nunique() # 1115
1115 * 942 # 1050330
df.date.nunique() # 942
df.store.nunique() # 1115

In [ ]:
# tester = train.merge(stores.merge(store_states.merge(state_names, on='State'), on='Store'), on='Store')
# Still left: google, weather
# len(train.columns) # 9
#tester = train.merge(tester)
#len(train.Store)
#train.Store.nunique()
# tester
len(sorted(weather.file.unique())) # 16
#len(sorted(tester.StateName.unique())) #12
# tester.head()
#print(sorted(tester.StateName.unique())) #12 missing Brandenburg, Bremen, MecklenburgVorpommern, Saarland
(sorted(weather.file.unique())) # 16
#sorted(weather.columns)#()
# len(tester.State.unique()) #12
#len(tester.StateName.unique()) #12


# This is the command that led to a runaway process
# tester.merge(weather, left_on='StateName', right_on='file').head()

# stores2, store_states, state_names, weather2 - only left google and train
# tester = (google.merge(stores2.merge(store_states.merge(state_names, on='State'), on='Store')).merge(weather2, left_on='StateName', right_on='file')
tester.isnull().sum()

In [ ]:
print(google.head())
len(google.file.unique()) # 14
len(tester.State.unique()) # 12
len(tester.StateName.unique()) # 12
len(tester.file.unique()) # 12
sorted(weather.file.unique())
# ['BadenWuerttemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'MecklenburgVorpommern', 'Niedersachsen', 'NordrheinWestfalen', 'RheinlandPfalz', 'Saarland', 'Sachsen', 'SachsenAnhalt', 'SchleswigHolstein', 'Thueringen']
sorted(google.file.unique())
# ['Rossmann_DE', 'Rossmann_DE_BE', 'Rossmann_DE_BW', 'Rossmann_DE_BY', 'Rossmann_DE_HE', 'Rossmann_DE_HH', 'Rossmann_DE_NI', 'Rossmann_DE_NW', 'Rossmann_DE_RP', 'Rossmann_DE_SH', 'Rossmann_DE_SL', 'Rossmann_DE_SN', 'Rossmann_DE_ST', 'Rossmann_DE_TH']
state_names.head()
len(state_names.State.unique()) # 16
(state_names.State.unique()) # 16
# 'BW', 'BY', 'BE', 'BB', 'HB', 'HH', 'HE', 'MV', 'HB,NI', 'NW', 'RP', 'SL', 'SN', 'ST', 'SH', 'TH']
# len(state_names.StateName.unique()) # 16
(state_names.StateName.unique()) # 16
# ['BadenWuerttemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'MecklenburgVorpommern', 'Niedersachsen', 'NordrheinWestfalen', 'RheinlandPfalz', 'Saarland', 'Sachsen', 'SachsenAnhalt', 'SchleswigHolstein', 'Thueringen'],
state_names
google2 = google.copy()
google2['State'] = google2.file.str[-2:]
len(google2.State.unique()) # 14
len(google.file.unique()) #14

sorted(google2.State.unique()) # 14
# ['BE', 'BW', 'BY', 'DE', 'HE', 'HH', 'NI', 'NW', 'RP', 'SH', 'SL', 'SN', 'ST', 'TH']
#sorted(google.file.unique()) # 14
#['Rossmann_DE',
# 'Rossmann_DE_BE',
# 'Rossmann_DE_BW',
# 'Rossmann_DE_BY',
# 'Rossmann_DE_HE',
# 'Rossmann_DE_HH',
# 'Rossmann_DE_NI',
# 'Rossmann_DE_NW',
# 'Rossmann_DE_RP',
# 'Rossmann_DE_SH',
# 'Rossmann_DE_SL',
# 'Rossmann_DE_SN',
# 'Rossmann_DE_ST',
# 'Rossmann_DE_TH']
sorted(state_names.StateName.unique()) # 16


In [ ]:
sorted(stores.Store.values) == list(range(1,1116)) # True
# stores.CompetitionOpenSinceMonth.hist()
#stores[stores.CompetitionOpenSinceYear>2000].CompetitionOpenSinceYear.hist()
stores.CompetitionOpenSinceYear.mean() # 2008
#stores[stores.CompetitionOpenSinceYear>1980].CompetitionOpenSinceYear.mean() # 2008
stores[stores.CompetitionOpenSinceYear>2000].CompetitionOpenSinceYear.mean() # 2009
stores.CompetitionOpenSinceMonth.mean() # 7
stores.describe()
#stores.Promo2.unique() # Binary, roughly half of stores have Promo2
#train.Store.nunique() #1115
len(stores[stores.Promo2 == 0]) #544
len(stores[stores.Promo2SinceWeek.isnull()]) #544
len(stores[stores.Promo2SinceYear.isnull()]) #544
stores[stores.Promo2SinceWeek.notnull()].head(15)
stores.PromoInterval.unique() # Either starts in Jan, starts in Feb, or starts in Mar, with quarterly restarts
#stores.info()
#stores.Promo2SinceWeek.hist()
#weather.info() # look out for Max_vis, Mean_vis, Min_vis, max_gust, cloudcover, events
weather.Events.describe()
#weather.Max_VisibilityKm.mean() # 24.0576
#weather.Min_VisibilitykM.mean()  # 7.0252
#weather.Mean_VisibilityKm.mean() # 12.2398
#24.0576-7.0252 # 17.032
#(weather.Max_VisibilityKm >= weather.Min_VisibilitykM).all()
#weather[(weather.Max_VisibilityKm.notnull()) & (weather.Min_VisibilitykM.notnull()) & (weather.Max_VisibilityKm < weather.Min_VisibilitykM)] # 15,459
#len(weather) # 15,840
weather.Events.value_counts()
len(weather[weather.Events.isnull()]) #3591
len(weather) #15840
weather.Events.describe() #11889
11889+3591
train.info()

In [ ]:
google.info()
google.describe()
# google.week.min() # 2012-12-02 - 2012-12-08
# google.week.max() # 2015-09-27 - 2015-10-03
# google.file.nunique() # 14
# google.week.nunique() # 148 = total file size / 14
sorted(google.file.unique())

# train.Date.min()  # 2013-01-01
# train.Date.max()  # 2015-07-31
# len(train) # 1,017,209
# train.info()

In [ ]:
# state_names.columns # StateName, State
# store_states.columns # State
# weather.columns # file
# google.columns # file

# state_names['tuple'] = [tuple(x) for x in state_names.values]
# state_names.tuple
# (BadenWuerttemberg, BW)
# (Bayern, BY)
# (Berlin, BE)
# (Brandenburg, BB)
# (Bremen, HB)
# (Hamburg, HH)
# (Hessen, HE)
# (MecklenburgVorpommern, MV)
# (Niedersachsen, HB,NI)
# (NordrheinWestfalen, NW)
# (RheinlandPfalz, RP)
# (Saarland, SL)
# (Sachsen, SN)
# (SachsenAnhalt, ST)
# (SchleswigHolstein, SH)
# (Thueringen, TH)

#store_states.State.nunique() # 12
#sorted(store_states.State.unique()) # 12

#store_states.State.nunique() # 12
#sorted(store_states.State.unique()) # 12
# ['BE', 'BW', 'BY', 'HB,NI', 'HE', 'HH', 'NW', 'RP', 'SH', 'SN', 'ST', 'TH']
# So store states don't include Saarland (SL) and 'HB,NI' should be considered 'NI' for google purposes
# store_states has 1115 rows, one for each store, and 12 states

weather.file.nunique() # 16
sorted(weather.file.unique()) # 16
# ['BadenWuerttemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'MecklenburgVorpommern',
#  'Niedersachsen', 'NordrheinWestfalen', 'RheinlandPfalz', 'Saarland', 'Sachsen', 'SachsenAnhalt',
#  'SchleswigHolstein', 'Thueringen']

len(google.file.unique()) # 14
sorted(google.file.unique()) # Take the last two and get
# ['BE', 'BW', 'BY', 'DE', 'HE', 'HH', 'NI', 'NW', 'RP', 'SH', 'SL', 'SN', 'ST', 'TH']
# Note 'DE' - is it a catchall for the others?  Or should others be considered a NaN?